In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.metrics import (
    f1_score,
    precision_recall_fscore_support,
    roc_auc_score,
    roc_curve,
    auc,
    precision_recall_curve,
    confusion_matrix
)
from catboost import Pool, CatBoostClassifier
import shap
shap.initjs()
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from var import DATA_OUT, IMAGE_OUT, FORECAST_HOURS_IN_ADVANCE

In [ ]:
df = pd.read_pickle(Path(DATA_OUT, 'df_dataset.pickle'))

In [ ]:
X = df[
    [
        'f_107_adj',
        'solar_zenith_angle',
        'newell',
#         'bz',
#         'vx',
#         'rho',
    ]
].copy()

y = df[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'].copy()

In [ ]:
HOURS_LAG = 4

X['newell_lagged'] = X['newell'].shift(2*HOURS_LAG)

In [ ]:
static_params = {
    "eval_metric": 'F1',
    "random_seed": 42,
    "auto_class_weights": "SqrtBalanced",
    "od_type": "Iter",
    "use_best_model": True,
    "has_time": True,
    "od_wait": 200,
}

In [ ]:
X_train, y_train = X.loc['2014':'2021-06'].copy(), y.loc['2014':'2021-06'].copy()
X_test, y_test = X.loc['2021-07':].copy(), y.loc['2021-07':].copy()

In [ ]:
model = CatBoostClassifier(
    loss_function='Logloss',
    iterations=2_000,
    **static_params,
)

model = model.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test),
    silent=True,
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
p, r, f, _ = precision_recall_fscore_support(y_test, y_pred)

In [ ]:
print(f'{p[1].round(3)} precision | {r[1].round(3)} recall | {f[1].round(3)} F1-score')

In [ ]:
shap_values = model.get_feature_importance(
    Pool(X_test, label=y_test),
    type="ShapValues",
)

shap_values = shap_values[:,:-1]

In [ ]:
shap.summary_plot(
    shap_values,
    X_test,
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

conf_matrix = confusion_matrix(y_test, y_pred, normalize='true')

plt.figure(figsize=(8, 8))

sns.set(font_scale=1.5)

sns.heatmap(
    conf_matrix,
    annot=True,
    fmt='.2f',
    cmap='Blues',
    cbar=False,
    xticklabels=['TID not predicted', 'TID predicted'],
    yticklabels=['TID doesn\'t occur', 'TID occurs'],
)

# plt.savefig(
#     Path(IMAGE_OUT, f'confusion_matrix.png', dpi=500, bbox_inches='tight')
# )

plt.show()